In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov5nu.pt')  # load an official model

# Export the model
# model.export(format='tflite', optimize=True, half=False, int8=False, simplify=True)

In [23]:
import torch, torch.nn as nn
from torch.nn.utils import prune
from ultralytics import YOLO

model_base = YOLO("yolov5nu.pt")


def sparsity(model):
    # Return global model sparsity
    a, b = 0, 0
    for p in model.parameters():
        a += p.numel()
        b += (p == 0).sum()
    return b / a

pruning_param = 0.9

for name, m in model_base.model.named_modules():
    if isinstance(m, nn.Conv2d) or isinstance(m, torch.nn.Linear):
        prune.l1_unstructured(m, name='weight', amount=pruning_param)  # prune
        prune.remove(m, 'weight')  # make permanent
print(f'Model pruned to {sparsity(model.model):.3g} global sparsity')

ckpt = {

            'model': model_base.model,
            'train_args': {},  # save as dict
}

prune_path = f'yolov5nu_pruned{pruning_param * 100}.pt'
torch.save(ckpt, prune_path)

model = YOLO(prune_path)



Model pruned to 0.896 global sparsity


In [24]:
model.export(format='tflite')

Ultralytics YOLOv8.1.28 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Core(TM) i7-1065G7 1.30GHz)
YOLOv5n summary (fused): 193 layers, 2649200 parameters, 0 gradients, 7.7 GFLOPs

PyTorch: starting from 'yolov5nu_pruned90.0.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (10.4 MB)

TensorFlow SavedModel: starting export with tensorflow 2.13.1...

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.8s, saved as 'yolov5nu_pruned90.0.onnx' (10.4 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.17.5...
TensorFlow SavedModel: export success ✅ 51.3s, saved as 'yolov5nu_pruned90.0_saved_model' (26.1 MB)

TensorFlow Lite: starting export with tensorflow 2.13.1...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolov5nu_pruned90.0_saved_model/yolov5nu_pruned90.0_float32.tflite' (10.4 MB)

Export complete (52.7s)
Results saved to /home/danylo/Desktop/mesh/univ/corusova
Predict:        

'yolov5nu_pruned90.0_saved_model/yolov5nu_pruned90.0_float32.tflite'